<a href="https://colab.research.google.com/github/skolix15/Machine_Learning_2025/blob/main/Tefas_Exercise_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SVM Classifier Class

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import time
from datetime import datetime
import pandas as pd
from itertools import product


# Used for classification => predicting discrete labels (e.g., cat vs dog, spam vs not spam).
# In classification, a higher C => fewer misclassifications but risk of overfitting.
class SVMClassifier:

    def __init__(self):

        # Hyperparameter grid
        self.param_grid = {
            "C": [0.1, 8, 50, 100],
            # "kernel": ["rbf", "poly", "linear"],
            # "kernel": ["rbf", "linear"],
            # "kernel": ["rbf"],
            "kernel": ["linear"],
            "gamma": ["scale", "auto"],
            # "gamma": ["auto"],
            "decision_function_shape": ["ovr", "ovo"],
            # "decision_function_shape": ["ovr"],
            # "decision_function_shape": ["ovo"],
        }
        self.results = []
        self.__model = []

    def get_model(self):
        return self.__model

    @staticmethod
    def count_correct_incorrect(y_true, y_pred):

        # Convert to numpy arrays
        y_true = np.array(y_true)
        y_pred = np.array(y_pred)

        # Ensure boolean arrays
        correct_mask = np.equal(y_true, y_pred)  # same as y_true == y_pred
        incorrect_mask = ~correct_mask

        correct = np.sum(correct_mask)
        incorrect = np.sum(incorrect_mask)

        return correct, incorrect

    def evaluate_all_models(self, x_train, x_test, y_train, y_test):

        # Create all parameter combinations
        keys = list(self.param_grid.keys())
        combinations = list(product(*self.param_grid.values()))

        print(f"Evaluating {len(combinations)} models...\n")

        best_test_accuracy = -1

        for combo in combinations:

            params = dict(zip(keys, combo))
            print(f"[MODEL] Model with parameters {params}\n")

            model = SVC(**params)

            # Training
            print(f"[TRAINING] Start")
            start_train = time.time()
            model.fit(x_train, y_train)
            train_time_minutes = (time.time() - start_train)/60
            print(f"[TRAINING] Finished in {train_time_minutes:.2f} minutes\n")

            # Testing
            print(f"[TESTING] Start")
            start_test = time.time()
            y_train_pred = model.predict(x_train)
            y_test_pred = model.predict(x_test)
            test_time_minutes = (time.time() - start_test)/60
            print(f"[TESTING] Finished in {test_time_minutes:.2f} minutes\n")

            # Accuracy
            print("[ACCURACIES] Calculate")
            train_accuracy = accuracy_score(y_train, y_train_pred)
            test_accuracy = accuracy_score(y_test, y_test_pred)
            print(f"[ACCURACIES] calculated => Train Acc: {train_accuracy:.2f} | Test Acc: {test_accuracy:.2f}\n")

            # Correct / incorrect counts
            print("[PREDICTIONS] Calculate")
            train_correct, train_incorrect = self.count_correct_incorrect(y_train, y_train_pred)
            test_correct, test_incorrect = self.count_correct_incorrect(y_test, y_test_pred)
            print(f"[PREDICTIONS] Calculated (Correct/Incorrect) => Train ({train_correct}/{train_incorrect}) | Test ({test_correct}/{test_incorrect})\n")

            # Divider
            print(f"{150 * '-'}\n")

            # Add data in results list
            self.results.append({
                **params,
                "train_accuracy": train_accuracy,
                "test_accuracy": test_accuracy,
                "train_time": train_time_minutes,
                "test_time": test_time_minutes,
                "train_correct": train_correct,
                "train_incorrect": train_incorrect,
                "test_correct": test_correct,
                "test_incorrect": test_incorrect
            })

            # Keep track of best model
            if test_accuracy > best_test_accuracy:

                best_test_accuracy = test_accuracy

                # Store the best model along with extra info
                self.__model = {
                    "model": model,
                    "train_accuracy": train_accuracy,
                    "test_accuracy": test_accuracy,
                    "train_time": train_time_minutes,
                    "test_time": test_time_minutes,
                    "train_correct": train_correct,
                    "train_incorrect": train_incorrect,
                    "test_correct": test_correct,
                    "test_incorrect": test_incorrect,
                    "params": params,
                    "y_test_pred": y_test_pred
                }

        # Store data to file
        # Convert to DataFrame
        df = pd.DataFrame(self.results)
        # Build timestamped filename
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"svm_full_results_{timestamp}.csv"
        df.to_csv(filename, index=False)
        print(f"Saved all model results to {filename}\n")

        return df

# Data Helper Class

In [ ]:
from tensorflow.keras.datasets import cifar10
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from scipy.io import loadmat


class DataHelper:

    @staticmethod
    def load_cifar10_data():

        # Load data from CIFAR10
        print("[LOAD CIFAR10] Start...")
        (x_train, y_train), (x_test, y_test) = cifar10.load_data()
        print("[LOAD CIFAR10] Finished\n")

        print(f"Training set: {x_train.shape}")
        print(f"Testing set: {x_test.shape}\n")

        return (x_train, y_train), (x_test, y_test)

    @staticmethod
    def custom_load_mat(path):
        data = loadmat(path)
        x = data['X']  # shape: (32, 32, 3, N)
        y = data['y'].squeeze()  # shape: (N,)

        # Move images to (N, 32, 32, 3)
        x = np.transpose(x, (3, 0, 1, 2))

        # SVHN uses label "10" to represent "0"
        y[y == 10] = 0
        return x, y

    @staticmethod
    def load_svhn_data():

        # Load data from CIFAR10
        print("[LOAD SVHN] Start...")

        # Load all sets
        x_train, y_train = DataHelper.custom_load_mat("./svhn_data/train_32x32.mat")
        x_test, y_test = DataHelper.custom_load_mat("./svhn_data/test_32x32.mat")
        # x_extra, y_extra = DataHelper.custom_load_mat("./svhn_data/extra_32x32.mat")

        # Normalize to [0, 1]
        x_train = x_train.astype("float32") / 255.0
        x_test = x_test.astype("float32") / 255.0

        print("[LOAD SVHN] Finished\n")

        print(f"Training set: {x_train.shape}")
        print(f"Testing set: {x_test.shape}\n")

        return (x_train, y_train), (x_test, y_test)

    @staticmethod
    def minimize_samples(x_train, x_test, y_train, y_test, max_train_samples=4000, max_test_samples=500):

        print("[MINIMIZE SAMPLES] Start...")
        np.random.seed(0)
        train_idx = np.random.choice(len(x_train), max_train_samples, replace=False)
        test_idx = np.random.choice(len(x_test), max_test_samples, replace=False)
        print("[MINIMIZE SAMPLES] Finished\n")

        return x_train[train_idx], x_test[test_idx], y_train[train_idx], y_test[test_idx]

    # Flatten images to vectors
    @staticmethod
    def flatten_samples(given_x_train, given_x_test):
        x_train = given_x_train.reshape(given_x_train.shape[0], -1)
        x_test = given_x_test.reshape(given_x_test.shape[0], -1)
        return x_train, x_test

    # Remove extra label dimension  (shape becomes (N,) instead of (N,1))
    @staticmethod
    def ravel_samples(given_y_train, given_y_test):
        y_train = given_y_train.ravel()
        y_test = given_y_test.ravel()
        return y_train, y_test

    @staticmethod
    def pca_samples(given_x_train, given_x_test):

        print("[PCA] Apply...")
        pca = PCA(n_components=0.90)
        x_train = pca.fit_transform(given_x_train)
        x_test = pca.transform(given_x_test)
        print("[PCA] Applied\n")

        print(f"Training set after PCA: {x_train.shape}")
        print(f"Testing set after PCA: {x_test.shape}\n")

        return x_train, x_test

    @staticmethod
    def flatten_and_pca_samples(given_x_train, given_x_test):

        # Flatten images to vectors before PCA
        x_train, x_test = DataHelper.flatten_samples(given_x_train, given_x_test)

        # Execute PCA in samples
        x_train, x_test = DataHelper.pca_samples(x_train, x_test)

        return x_train, x_test

    @staticmethod
    def show_grid_examples(indices, images, y_true, y_pred, title):

        class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

        plt.figure(figsize=(12, 4))

        for idx, sample_idx in enumerate(indices):
            img = images[sample_idx].reshape(32, 32, 3)

            true_label = class_names[int(y_true[sample_idx])]
            pred_label = class_names[int(y_pred[sample_idx])]

            plt.subplot(1, len(indices), idx + 1)
            plt.imshow(img.astype(np.uint8))
            plt.axis("off")
            plt.title(f"T: {true_label}\nP: {pred_label}")

        plt.suptitle(title, fontsize=14)
        plt.tight_layout()
        plt.show()

    @staticmethod
    def display_grid_with_random_indices(indexes, size, original_min_x_test, min_y_test, class_y_test_pred, tile):
        random_indexes = np.random.choice(indexes, size=size, replace=False)
        DataHelper.show_grid_examples(
            random_indexes,
            original_min_x_test, min_y_test, class_y_test_pred,
            tile
        )

    @staticmethod
    def random_visualization(correct_size, incorrect_size, model_info, desired_y_test, original_x_test):

        class_y_test_pred = model_info["y_test_pred"]

        # Find correct and incorrect indices
        correct_idx = np.where(class_y_test_pred == desired_y_test)[0]
        incorrect_idx = np.where(class_y_test_pred != desired_y_test)[0]

        # Random correct classifications
        DataHelper.display_grid_with_random_indices(
            correct_idx, correct_size,
            original_x_test, desired_y_test, class_y_test_pred,
            f"{correct_size} Correct Classifications"
        )

        # Random incorrect classifications
        DataHelper.display_grid_with_random_indices(
            incorrect_idx, incorrect_size,
            original_x_test, desired_y_test, class_y_test_pred,
            f"{incorrect_size} Incorrect Classifications"
        )


# Main

In [ ]:
from datetime import datetime


def main():

    minimize_samples = True

    # Load data from CIFAR10
    (x_train, y_train), (x_test, y_test) = DataHelper.load_cifar10_data()
    # (x_train, y_train), (x_test, y_test) = DataHelper.load_svhn_data()

    if minimize_samples:

        # Minimize samples
        min_x_train, min_x_test, min_y_train, min_y_test = DataHelper.minimize_samples(
            x_train, x_test, y_train, y_test,
            max_train_samples=5000,
            max_test_samples=835
        )

        desired_x_train, desired_x_test, desired_y_train, desired_y_test = min_x_train, min_x_test, min_y_train, min_y_test

    else:

        desired_x_train, desired_x_test, desired_y_train, desired_y_test = x_train, x_test, y_train, y_test

    # # Save original minimized x test samples
    # original_x_test = desired_x_test.copy()

    # Flatten and PCA
    desired_x_train, desired_x_test = DataHelper.flatten_and_pca_samples(given_x_train=desired_x_train, given_x_test=desired_x_test)

    # Remove extra label dimension  (shape becomes (N,) instead of (N,1))
    desired_y_train, desired_y_test = DataHelper.ravel_samples(given_y_train=desired_y_train, given_y_test=desired_y_test)

    # Create svm classifier
    svm_classifier = SVMClassifier()
    df = svm_classifier.evaluate_all_models(
        desired_x_train, desired_x_test,
        desired_y_train, desired_y_test
    )

    # Print data frame
    print("All model results:\n\n")
    print(df)

    # Get desired model information
    best_model_info = svm_classifier.get_model()

    # Print best model information
    print(f"\n\nBest model:\n\n{best_model_info}\n")

    # # Random visualization
    # DataHelper.random_visualization(3, 3, best_model_info, desired_y_test, original_x_test)

if __name__ == '__main__':

    # Start datetime
    start_datetime = datetime.now()
    print(f"\n[TIME] Start datetime: {start_datetime.strftime('%d/%m/%Y %H:%M:%S')}\n")

    # Execute main
    main()

    # End datetime
    end_datetime = datetime.now()
    print(f"[TIME] End datetime: {end_datetime.strftime('%d/%m/%Y %H:%M:%S')}\n")

    # Execution time
    execution_time_minutes = (end_datetime - start_datetime).total_seconds()/60
    print(f"[TIME] Execution time: {execution_time_minutes:.2f} minutes\n")